In [18]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.svm import SVC
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
import optuna
warnings.filterwarnings("ignore")

In [ ]:
# Brain tumor detection using SVM
data = pd.read_csv('brain_tumor_dataset.csv')
data = data.drop(columns=["Patient_ID", "Symptom_1", "Symptom_2", "Symptom_3"])
# data = data.head(100)
data = pd.get_dummies(data, columns=["Gender", "Location", "Histology", "Stage"])
data["Radiation_Treatment"] = data["Radiation_Treatment"].replace({"Yes": True, "No": False})
data["Surgery_Performed"] = data["Surgery_Performed"].replace({"Yes": True, "No": False})
data["Chemotherapy"] = data["Chemotherapy"].replace({"Yes": True, "No": False})
data["Family_History"] = data["Family_History"].replace({"Yes": True, "No": False})
data["Follow_Up_Required"] = data["Follow_Up_Required"].replace({"Yes": True, "No": False})
data["MRI_Result"] = data["MRI_Result"].replace({"Positive": True, "Negative": False})
X = data.drop(columns=["Tumor_Type"])
y = data["Tumor_Type"]
y = y.replace({"Benign": 0, "Malignant": 1})
data

,Age,Tumor_Type,Tumor_Size,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,...,Location_Parietal,Location_Temporal,Histology_Astrocytoma,Histology_Glioblastoma,Histology_Medulloblastoma,Histology_Meningioma,Stage_I,Stage_II,Stage_III,Stage_IV
0,73,Malignant,5.375612,False,False,False,51.312579,0.111876,False,True,...,False,True,True,False,False,False,False,False,True,False
1,26,Benign,4.847098,True,True,True,46.373273,2.165736,True,True,...,True,False,False,True,False,False,False,True,False,False
2,31,Benign,5.588391,False,False,False,47.072221,1.884228,False,False,...,True,False,False,False,False,True,True,False,False,False
3,29,Malignant,1.436600,True,False,True,51.853634,1.283342,True,False,...,False,True,False,False,True,False,False,False,False,True
4,54,Benign,2.417506,False,False,True,54.708987,2.069477,False,True,...,True,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,57,Malignant,3.643067,True,False,True,71.212734,0.657977,True,False,...,False,True,False,False,True,False,False,False,False,True
96,28,Malignant,0.739781,False,True,True,58.184343,0.199555,True,False,...,True,False,False,False,False,True,False,False,False,True
97,34,Malignant,7.429725,True,True,True,91.259050,0.951945,False,True,...,False,True,True,False,False,False,False,False,True,False
98,34,Malignant,9.540553,True,True,True,79.082038,2.640211,False,True,...,False,False,True,False,False,False,False,False,True,False


In [23]:
scaler = StandardScaler()
features_to_scale = ["Age", "Tumor_Size", "Survival_Rate", "Tumor_Growth_Rate"]
X[features_to_scale] = scaler.fit_transform(X[features_to_scale])
X

,Age,Tumor_Size,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required,...,Location_Parietal,Location_Temporal,Histology_Astrocytoma,Histology_Glioblastoma,Histology_Medulloblastoma,Histology_Meningioma,Stage_I,Stage_II,Stage_III,Stage_IV
0,1.424273,-0.046646,False,False,False,-1.213105,-1.473296,False,True,True,...,False,True,True,False,False,False,False,False,True,False
1,-1.362606,-0.243556,True,True,True,-1.502782,0.763138,True,True,True,...,True,False,False,True,False,False,False,True,False,False
2,-1.066129,0.032630,False,False,False,-1.461791,0.456606,False,False,False,...,True,False,False,False,False,True,True,False,False,False
3,-1.184720,-1.514212,True,False,True,-1.181373,-0.197695,True,False,False,...,False,True,False,False,True,False,False,False,False,True
4,0.297662,-1.148754,False,False,True,-1.013914,0.658323,False,True,True,...,True,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.475548,-0.692144,True,False,True,-0.046011,-0.878650,True,False,True,...,False,True,False,False,True,False,False,False,False,True
96,-1.244015,-1.773828,False,True,True,-0.810093,-1.377823,True,False,True,...,True,False,False,False,False,True,False,False,False,True
97,-0.888244,0.718660,True,True,True,1.129654,-0.558551,False,True,False,...,False,True,True,False,False,False,False,False,True,False
98,-0.888244,1.505096,True,True,True,0.415503,1.279791,False,True,True,...,False,False,True,False,False,False,False,False,True,False


In [4]:
'''
k_folds = 10
svm = SVC(kernel='linear', C=1.0, probability=True, random_state=42)
scores = cross_val_score(svm, X, y, scoring='roc_auc_ovr', cv=k_folds)
print(f"SVM Classification ROC AUC over {k_folds} folds: {np.mean(scores):.4f} ± {np.std(scores):.4f}")
'''

'\nk_folds = 10\nsvm = SVC(kernel=\'linear\', C=1.0, probability=True, random_state=42)\nscores = cross_val_score(svm, X, y, scoring=\'roc_auc_ovr\', cv=k_folds)\nprint(f"SVM Classification ROC AUC over {k_folds} folds: {np.mean(scores):.4f} ± {np.std(scores):.4f}")\n'

In [25]:
def objective(trial):
    C = trial.suggest_loguniform('C', 0.01, 10)
    svm = SVC(kernel='rbf', C=C, probability=True, random_state=42)
    
    k_folds = 10
    cv = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    scores = cross_val_score(svm, X, y, scoring='roc_auc_ovr', cv=cv, n_jobs=-1)

    return np.mean(scores)

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=10)

print("Best C:", study.best_params['C'])
print(f"Best CV ROC AUC: {study.best_value:.4f}")



[I 2025-10-27 17:23:57,766] A new study created in memory with name: no-name-de8bade4-712b-4097-97bc-619c36423231
[I 2025-10-27 17:23:57,796] Trial 0 finished with value: 0.5720714285714286 and parameters: {'C': 0.02061470450043262}. Best is trial 0 with value: 0.5720714285714286.
[I 2025-10-27 17:23:57,823] Trial 1 finished with value: 0.574154761904762 and parameters: {'C': 0.9774988742952144}. Best is trial 1 with value: 0.574154761904762.
[I 2025-10-27 17:23:57,837] Trial 2 finished with value: 0.5639047619047621 and parameters: {'C': 0.1520734192579403}. Best is trial 1 with value: 0.574154761904762.
[I 2025-10-27 17:23:57,851] Trial 3 finished with value: 0.5639047619047621 and parameters: {'C': 0.15853796982804713}. Best is trial 1 with value: 0.574154761904762.
[I 2025-10-27 17:23:57,876] Trial 4 finished with value: 0.5720714285714286 and parameters: {'C': 0.027516061183271143}. Best is trial 1 with value: 0.574154761904762.
[I 2025-10-27 17:23:57,902] Trial 5 finished with va

Best C: 3.5174886780603307
Best CV ROC AUC: 0.5831
